# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

### Query

```sql
SELECT
  EXTRACT(YEAR
  FROM
    order_approved_at) AS order_approved_year,
  COUNT(*) AS total
FROM
  `dbt-shop-luizlopes.dtb_shop.orders`
GROUP BY
  order_approved_year
ORDER BY 
  order_approved_year;
```

### Comando

`dbt run --models orders_by_year`

### Resultado

![Orders By Year](imagens/orders_by_year_gcp.png)

2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

## Query

```sql
SELECT
  customers.customer_state,
  COUNT(*) AS total
FROM
  `dbt-shop-luizlopes.dtb_shop.customers` customers
INNER JOIN
  `dbt-shop-luizlopes.dtb_shop.orders` orders
ON
  orders.customer_id = customers.customer_id
GROUP BY
  customers.customer_state
ORDER BY
  customers.customer_state
```

## Comando

`dbt run --models orders_by_customer_state`

## Resultado


![Orders by customer state](imagens/orders_by_customer_state.png)


3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

## Query

```sql
SELECT
  customer_state, total
FROM
  `dbt-shop-luizlopes.dtb_shop.orders_by_customer_state`
WHERE
  total is not null and total >= CAST('{{ var("number_of_orders") }}' AS integer)
ORDER BY customer_state
```

## Comando

`dbt run --models orders_by_customer_state_with_filter --vars "{'number_of_orders': '5000'}"`

## Resultado

![Orders by customer state with filter](imagens/orders_by_customer_state_with_filter.png)

4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

## Query

```sql
SELECT
  count(*) as total,
  CASE
    WHEN customer_state IN ('GO', 'MT', 'MS', 'DF') THEN 'Centro-Oeste'
    WHEN customer_state IN ('MA', 'PI', 'RN', 'CE', 'PB', 'BA', 'PE', 'AL', 'SE') THEN 'Nordeste'
    WHEN customer_state IN ('AM', 'AC', 'RO', 'RR', 'AP', 'PA', 'TO') THEN 'Norte'
    WHEN customer_state IN ('ES', 'MG', 'RJ', 'SP') THEN 'Sudeste'
    WHEN customer_state IN ('PR', 'RS', 'SC') THEN 'Sul'
  END AS customer_region
FROM
  `dbt-shop-luizlopes.dtb_shop.customers`
GROUP BY customer_region
```

## Comando

`dbt run --models customers_by_region`

## Resultado

![Customers by region](imagens/customers_by_region.png)


5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

## Query

```sql
SELECT
  *,
  (price + freight_value) AS total_price
FROM
  `dbt-shop-luizlopes.dtb_shop.items`
```

## Comando

`dbt run --models items_costs`

## Resultado

![Items costs](imagens/items_costs.png)


6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

## Query

```sql
SELECT
  round(order_total_price.order_total_price, 2) as order_total_price,
  orders.*
FROM
  `dbt-shop-luizlopes.dtb_shop.orders` orders
INNER JOIN (
  SELECT
    items_costs.order_id,
    SUM(items_costs.total_price) AS order_total_price
  FROM
    `dbt-shop-luizlopes.dtb_shop.items_costs` items_costs
  GROUP BY
    items_costs.order_id) order_total_price
ON
  order_total_price.order_id = orders.order_id
WHERE
  orders.order_status = 'shipped'
```

## Comando

`dbt run --models orders_shipped_costs`

## Resultado

![Order shipped costs](imagens/order_shipped_costs.png)
